In [5]:
import asyncio, aiohttp
import nest_asyncio
import yarl
nest_asyncio.apply()

In [1]:
BPMLIST = ['IBA1C','IBA2R','IBA3R','IBB1R','IBB2R','IBC1R','IBC2R','IBD1R','IBD2R','IBE1R','IBE2R','IBE2L','IBE1L','IBD2L','IBD1L','IBC2L','IBC1L','IBB2L','IBB1L','IBA3L','IBA2L']
#BPMLIST_GOOD = ['IBA2R','IBA3R','IBB1R','IBB2R','IBC1R','IBC2R','IBD1R','IBD2R','IBE1R','IBE2R','IBE1L','IBD2L','IBD1L','IBC2L','IBC1L','IBB2L','IBB1L','IBA3L','IBA2L']
BPMLIST_GOOD = ['IBA2R','IBA3R','IBB1R','IBB2R','IBC1R','IBC2R','IBD1R','IBD2R','IBE1R','IBE2R','IBD2L','IBD1L','IBC2L','IBC1L','IBB2L','IBB1L','IBA3L','IBA2L']
BPMLIST_H, BPMLIST_V, BPMLIST_S   = ["N:"+i+"H" for i in BPMLIST], ["N:"+i+"V" for i in BPMLIST], ["N:"+i+"S" for i in BPMLIST]
BPMLIST_HV = BPMLIST_H + BPMLIST_V
BPMLIST_ALL = BPMLIST_H + BPMLIST_V + BPMLIST_S

In [ ]:
BPMLIST = ['IBA1C','IBA2R','IBA3R','IBB1R','IBB2R','IBC1R','IBC2R','IBD1R','IBD2R','IBE1R','IBE2R','IBE2L','IBE1L','IBD2L','IBD1L','IBC2L','IBC1L','IBB2L','IBB1L','IBA3L','IBA2L']
#BPMLIST_GOOD = ['IBA2R','IBA3R','IBB1R','IBB2R','IBC1R','IBC2R','IBD1R','IBD2R','IBE1R','IBE2R','IBE2L','IBE1L','IBD2L','IBD1L','IBC2L','IBC1L','IBB2L','IBB1L','IBA3L','IBA2L']
BPMLIST_GOOD = ['IBA2R','IBA3R','IBB2R','IBC1R','IBC2R','IBD1R','IBD2R','IBE1R','IBE2R','IBD2L','IBD1L','IBC2L','IBC1L','IBB2L','IBB1L','IBA3L','IBA2L'] #E1L looked bad
#BPMLIST_GOOD = ['IBA2R','IBA3R','IBB1R','IBB2R','IBC1R','IBC2R','IBD1R','IBD2R','IBE1R','IBE2R','IBE1L','IBD2L','IBD1L','IBC2L','IBC1L','IBB2L','IBB1L','IBA3L','IBA2L'];
BPMLIST_H, BPMLIST_V, BPMLIST_S   = ["N:"+i+"H" for i in BPMLIST], ["N:"+i+"V" for i in BPMLIST], ["N:"+i+"S" for i in BPMLIST]
BPMLIST_HG, BPMLIST_VG, BPMLIST_SG   = ["N:"+i+"H" for i in BPMLIST_GOOD], ["N:"+i+"V" for i in BPMLIST_GOOD], ["N:"+i+"S" for i in BPMLIST_GOOD]
BPMLIST_HV = BPMLIST_H + BPMLIST_V; BPMLIST_HVG = BPMLIST_HG + BPMLIST_VG
BPMLIST_ALL = BPMLIST_H + BPMLIST_V + BPMLIST_S; BPMLIST_ALLG = BPMLIST_HG + BPMLIST_VG + BPMLIST_SG

In [2]:
def decorate_names(bpmlist, kinds):
    resp = []
    if 'H' in kinds:
        resp += ["N:"+i+"H" for i in bpmlist]
    if 'V' in kinds:
        resp += ["N:"+i+"V" for i in bpmlist]
    if 'S' in kinds:
        resp += ["N:"+i+"S" for i in bpmlist]
    return resp

In [5]:
def getBPMdata(z, bpmlist):
    data = {};
    l = 2098;
    try:
        bpmlist = [b+'[]' if '[' not in b else b for b in bpmlist]
        r = z.readDeviceList(bpmlist)
        if isinstance(r, typeerr):
            print('BAAAAAAD')
            raise
        for i, k in enumerate(r):
            if i == 1:
                aqtime = r[k].getTime();
                units = r[k].getUnit()
            kname = k.split('[')[0]        
            try:
                data[kname] = np.array(r[k].doubleArray()).astype(np.double);
            except:
                print('{} failed!'.format(k), r[k].__class__, r[k].getMessage())
                #data[kname] = np.zeros(l)
                raise
        return (data, aqtime, units)
    finally:
        #z.enableSettings(False,1)
        pass    

In [6]:
def getBPMdataACL(s, bpmlist):     
    #print('t1',t1)
    bpmstring = ''
    try:
        for b in bpmlist:
            bpmstring += '{{{}}},'.format(b)
        #print(bpmstring[:-1])
        txt = "http://www-ad.fnal.gov/cgi-bin/acl.pl?acl=read+/row+/pendWait=3+devices=\'"+ bpmstring[:-1] + '\'+/all_elements'
        #print(txt)
        #r = s.get(txt)
        from urllib.request import urlopen
        resp = urlopen(txt)
        r = resp.read().decode("ascii")
        t1 = datetime.datetime.utcnow().timestamp()
        if "DPM_PEND" in r:
            print('ERRORROROROROROROROOROR')
        #print(repr(r))
        if 'mm' not in r and 'raw' not in r:
            raise Exception()
        #split_text = r.strip().replace('mm','raw').split('raw')  
        split_text = r.strip().split('\n')
        #print(split_text)
        data = {}
        for s in filter(None, split_text):
            spl = s.strip().split()
            #print(s, spl)
            data[spl[0][:-3]] = np.array([float(v) for v in spl[2:-1]])
        #current = -float(re.findall('[-,\.,\d]+', r.text)[0])
        #time.sleep(0.01)
        return (data, t1, 'unknown')
    except Exception as e: 
        print(s, e, sys.exc_info())
        raise

In [7]:
async def fetch_task(session, url):
    url = yarl.URL(url, encoded=True)
    async with session.get(url) as response:
        return await response.read()

async def http_fetch_async(loop, urls, responses):
    connector = aiohttp.TCPConnector(limit=None, ssl=False)
    tasks = []

    # Fetch all responses within one Client session
    # keep connection alive for all requests.
    async with aiohttp.ClientSession(loop=loop, connector=connector) as session:
    #async with aiohttp.ClientSession(loop=loop) as session:
        for url in urls:
            task = asyncio.ensure_future(fetch_task(session, url))
            tasks.append(task)
            #await asyncio.sleep(0.01)
        responses = await asyncio.gather(*tasks,return_exceptions=True)
        # you now have all response bodies in this variable
        #print(responses)
        return responses
        
def getBPMdataACLAsync(s, bpmlist_initial): 
    #print('t1',t1)
    numlists = min(len(bpmlist_initial),5)
    bpmlists = [list(l) for l in np.array_split(bpmlist_initial,numlists)]
    bpmstrings = []
    for bpmlist in bpmlists:
        bpmstring = ''
        for b in bpmlist:
            bpmstring += '{{{}}},'.format(b)
            #print(bpmstring[:-1])
        bpmstring = "http://www-ad.fnal.gov/cgi-bin/acl.pl?acl=read+/row+/pendWait=1+devices=\'"+ bpmstring[:-1] + '\'+/all_elements'
#     for b in bpmlist:
#         base = "http://www-ad.fnal.gov/cgi-bin/acl.pl?acl=read+/row+/pendWait=3+"
#         bpmstring = base+'{}'.format(b)+"+/all_elements"
        bpmstrings.append(bpmstring)
    #print(bpmstrings)
    try:
        data = {}
        loop = asyncio.get_event_loop()
        #loop = asyncio.new_event_loop()
        #asyncio.set_event_loop(loop)
        #print('Loop:', loop, loop.is_running())
        #future = asyncio.ensure_future(http_fetch_async(loop, bpmstrings),loop=loop)
        #results = loop.run_until_complete(future)
        responses = []
        #task = loop.create_task(http_fetch_async(loop, bpmstrings, responses))
        responses = loop.run_until_complete(http_fetch_async(loop, bpmstrings, responses))
        #results = asyncio.gather(task)
        #print(responses)
        t1 = datetime.datetime.utcnow().timestamp()
        #loop.close()
        #results = asyncio.gather(future)
        for r in responses: 
            r = r.decode('ascii')
            #print(r)
            #print(results)
            #print(txt)
            #r = s.get(txt)          
            if 'mm' not in r and 'raw' not in r:
                raise Exception()
            split_text = r.strip().split('\n')
            #print(split_text)
            data_tmp = {}
            for s in filter(None, split_text):
                spl = s.strip().split()
                #print(spl[0])
                data_tmp[spl[0][:-3]] = np.array([float(v) for v in spl[2:-1]])
            data.update(data_tmp)
        return (data, t1, 'unknown')
    except Exception as e: 
        print(s, e, sys.exc_info())
        raise

In [8]:
def bpmDAQWrapperAsync(z, bpmattempts, faststate=False):     
    attemptgood = False; j = 0; t1 = time.time_ns();
    while not attemptgood and j < bpmattempts:         
        try:
            data = {}  
            print('Getting current data')
            if faststate:
                currents = get_devices_fast(['N:IBEAM@I','N:IBEASL@I'], setting=False)
                sl = currents['N:IBEASL@I']; dcct = currents['N:IBEAM@I']   
            else:
                currents = get_devices_fast(['N:IBEAM','N:IBEASL'], setting=False)
                sl = currents['N:IBEASL']; dcct = currents['N:IBEAM']   
            for r in [sl, dcct]:
                if isinstance(r, typeerr) and (r.getErrorNumber() != 0 or r.getFacilityCode() != 0):
                    print(r.getMessage())
                    print('Request failed, retrying')
                    continue  
            print('Currents: dcct {} | sl {}'.format(dcct,sl))            
            #sl, dcct = sl.doubleValue(), dcct.doubleValue()
            print('Getting ring state setpoints')
            if faststate:
                device_state = get_devices_fast([k+'@I' for k in knobs_to_save], setting=True)
            else:
                device_state = get_devices_fast(knobs_to_save, setting=True)
            print('Getting BPMs')
            blist_int = [v for b in blists for v in b]
            aqgood = False; k = 0;
            while not aqgood and k < 3:
                try:
                    (data, aqtime, units) = getBPMdataACLAsync(z, blist_int)
                    time.sleep(0.01)
                    aqgood = True
                except Exception as e: 
                    print('Readout retry #{}'.format(k))
                    k += 1  
                    time.sleep(0.01)
                    if k == 3: raise
            camdata = {}
            for (b,v) in dstrings.items():
                camdata[b] = list(dstrings[b])[-campoints:]
            attemptgood = True; t2 = time.time_ns()
            print('Readout done in {:.3f}s'.format((t2-t1)/1e9))  
        except Exception as e:
            j += 1
            print(e)
            print('BPM readout exception!')    
            raise e
    return (sl, dcct, data, aqtime, units, device_state, camdata)

In [9]:
def bpmDAQWrapper(z, bpmattempts, faststate=False):     
    attemptgood = False; j = 0; t1 = time.time_ns();
    while not attemptgood and j < bpmattempts:         
        try:
            data = {}  
            print('Getting current data')
            if faststate:
                currents = get_devices_fast(['N:IBEAM@I','N:IBEASL@I'], setting=False)
                sl = currents['N:IBEASL@I']; dcct = currents['N:IBEAM@I']   
            else:
                currents = get_devices_fast(['N:IBEAM','N:IBEASL'], setting=False)
                sl = currents['N:IBEASL']; dcct = currents['N:IBEAM']    
            for r in [sl, dcct]:
                if isinstance(r, typeerr) and (r.getErrorNumber() != 0 or r.getFacilityCode() != 0):
                    print(r.getMessage())
                    print('Request failed, retrying')
                    continue  
            print('Currents: dcct {} | sl {}'.format(dcct,sl))            
            #sl, dcct = sl.doubleValue(), dcct.doubleValue()
            print('Getting ring state setpoints')
            if faststate:
                device_state = get_devices_fast([k+'@I' for k in knobs_to_save], setting=True)
            else:
                device_state = get_devices_fast(knobs_to_save, setting=True)
            print('Getting BPMs')
            for blist in blists:
                aqgood = False; k = 0;
                while not aqgood and k < 3:
                    try:
                        (data_tmp, aqtime, units) = getBPMdataACL(z, blist)
                        time.sleep(0.01)
                        aqgood = True
                    except Exception as e: 
                        print('Readout retry #{}'.format(k))
                        k += 1  
                        time.sleep(0.01)
                        if k == 3: raise
                data.update(data_tmp)
            camdata = {}
            for (b,v) in dstrings.items():
                camdata[b] = list(dstrings[b])[-campoints:]
            attemptgood = True; t2 = time.time_ns()
            print('Readout done in {:.3f}s'.format((t2-t1)/1e9))  
        except Exception as e:
            j += 1
            print(e)
            print('BPM readout exception!')        
    return (sl, dcct, data, aqtime, units, device_state, camdata)

In [10]:
def bpmSaveWrapper(z, saverootdir='OCP_kicks'):        
    savedir = datetime.datetime.now().strftime("%Y_%m_%d")
    savedirfull = "Y:\\nkuklev\\_expData\\"+saverootdir+"\\{}".format(savedir)
    if not os.path.isdir(savedirfull):
        print('Creating new directory: {}'.format(savedirfull))
        os.makedirs(savedirfull)

    savetime = datetime.datetime.now().strftime("%Y%m%d-%H%M%S");
    filename = savetime;
    filenamefull = savedirfull+os.path.sep+'ocp_kicks_'+filename+'.hdf5'
    if os.path.exists(filenamefull):
        print('Warning - path {} exists, aborting'.format(filenamefull))
        raise Exception

    with h5py.File(filenamefull,'w', libver='latest') as f:
        for (k,v) in data.items():
            f.create_dataset(k,data=v,compression='gzip',compression_opts=9,shuffle=True,fletcher32=True)                
        #f.create_dataset('dcct_array',data=dcct_arr,compression='gzip',compression_opts=9,shuffle=True,fletcher32=True)

        stategr = f.create_group('state')
        for (k,v) in device_state.items():
            stategr.attrs[k] = v
        stategr.attrs['kickV'] = vkick
        stategr.attrs['kickH'] = hkick
        stategr.attrs['N:IBEAM'] = dcct
        stategr.attrs['N:IBEASL'] = sl

        camgr = f.create_group('cameras')
        for (b,v) in camdata.items():
            #print(b,v)
            camgr.create_dataset(b,data=[np.string_(s) for s in v],dtype=h5py.special_dtype(vlen=str),
                                 compression='gzip',compression_opts=9,shuffle=True,fletcher32=True)

        f.attrs['time_utcstamp'] = datetime.datetime.utcnow().timestamp();
        f.attrs['time_save'] = savetime
        f.attrs['time_acnet'] = aqtime
        f.attrs['time_run'] = time_run.strftime("%Y%m%d-%H%M%S")
        f.attrs['units'] = units
        f.attrs['kickV'] = vkick
        f.attrs['kickH'] = hkick
        f.attrs['IBEAM'] = dcct
        f.attrs['IBEASL'] = sl
        f.attrs['comment'] = comment
        f.attrs['datatype'] = 'BPM OCTOMON TBT KICKS V4'
        f.attrs['uuid'] = str(uuid.uuid4())
        f.attrs['run_uuid'] = str(run_uuid)

In [11]:
def kickWrapper(z, vkick, kickattempts=5):
    attemptgood = False; j = 0; t1 = time.time_ns();
    while not attemptgood and j < kickattempts:     
        print('Kick:', vkick, j)       
        for i in range(kickattempts):            
            try:
                print('Kick attempt {}'.format(i), end='')
                if vkick == 0:
                    z.enableSettings(True, 1)
                    r = z.setDevice('N&IBPSTATD','ARMOrbit')
                    if isinstance(r, typeerr) and (r.getErrorNumber() != 0 or r.getFacilityCode() != 0):
                        print(r.getMessage())
                        print('Orbit BPM request failed, retrying')
                        continue
                    z.enableSettings(False, 1)
                    time.sleep(2)
                    status = True
                else:
                    status = VKickChipFast(z, vkick)
                if status:
                    print('-> OK')
                    attemptgood = True; t2 = time.time_ns()
                    break
                else:
                    j += 1
                    print('-> FAILED')
            except Exception as e:
                j += 1
                print(e)
                print('Kick Exception!')
                z.enableSettings(False,2)            
                time.sleep(1)
                continue
        if not status:
            print('Kick failed')
            raise Exception            
    print('Kick done in {:.3f}s'.format((t2-t1)/1e9))

In [12]:
def VKickChipFast(z, kv):
    status = False
    fired = False
    kick_set = False
    kv = float(kv)
    if kv <= 0 or kv >= 1.5:
        print('Incorrect V kick magnitude - {} KV'.format(kv))
        return False
    
    try:  
        z.enableSettings(True, 5)
        # Close shutter
        if z.readDevice('N:LGXS.STATUS@I').isOn().value:
            print('Closing shutter')
            for i in range(10):
                r = z.setDevice('N&LGXS','OFF');
                print('N&LGXS', r)
                time.sleep(1) 
                r = z.readDevice('N|LGXS');
                print('N|LGXS', r)
                if not r.isOn().value:
                    print('Shutter closed!')
                    break
                else:
                    print('Shutter did not close - attempt #:', i)
                    if i == 10:
                        print('Aborting!')
                        raise Exception()
        else:
            print('Shutter is already closed')        
        time.sleep(0.01)
#         # Close shutter
#         if z.readDevice('N:ASTCDC.STATUS').isOn().value:
#             print('Turning off CDC')
#             for i in range(10):
#                 r = z.setDevice('N&ASTCDC','OFF');
#                 print('N&ASTCDC', r)
#                 time.sleep(0.3) 
#                 r = z.readDevice('N|ASTCDC');
#                 print('N|ASTCDC', r)
#                 if not r.isOn().value:
#                     print('CDC off!')
#                     break
#                 else:
#                     print('CDC did not turn off - attempt #:', i)
#                     if i == 10:
#                         print('Aborting!')
#                         raise Exception()
#         else:
#             print('CDC is already off')        
#         time.sleep(0.1)

        # Kicker 
        print('--Setting kicker')
        desired_kicker_setpt = kv
        kick_set = True;
        r = z.readDevice('N:IKPSV.STATUS')
        print('N:IKPSV.STATUS', r)
        if not r.isOn().value:
            raise Exception();
        if not r.isReady().value:
            z.setDevice('N&IKPSV','RESET')
            print('Kicker not ready - resetting!')
            time.sleep(0.5);
            r = z.readDevice('N:IKPSV.STATUS').isReady()
            print('New status:', r)
            if not r.value:
                raise Exception
        
        r = z.setDevice('N_IKPSV', desired_kicker_setpt)
        time.sleep(0.1);
        r = z.readDevice('N:IKPSV')
        print('N:IKPSV', r)
        r = r.doubleValue()
#         r = z.setDevice('N_IKPSVX', desired_kicker_setpt)
#         time.sleep(0.1);
#         r = z.readDevice('N_IKPSVX')
#         print('N_IKPSV', r)
#         r = r.doubleValue()
        if r < desired_kicker_setpt-0.03 or r > desired_kicker_setpt+0.03:
            raise Exception('Setpoing not within desired range')
        print('Kicker set - readback {}'.format(r))
        time.sleep(0.1);
                
        # Trigger chip thing
        print('--CHIP TRIGGER')
        r = z.setDevice('N_IBEAMR',0)
        
        print('--AWAIT A6')
        r = z.readDevice('N:LGA1DI@e,A6')
        print('N:LGA1DI@e,A6', r)    
        #time.sleep(5);

        status = True
    except Exception as e:
        print('ERROR!!!')
        print(e)

    finally:
        # Kicker restore
        if kick_set:
            print('--Restoring kicker readiness')
            attempts = 10
            try:
                r = z.readDevice('N:IKPSV.STATUS@I')
                print('N:IKPSV.STATUS', r)
            except Exception as e:
                raise e

            if not r.isOn().value:
                print('===KICKER IS NOT ON - SOMETHING IS WRONG===')
                raise Exception()
            else:
                print('Kicker still ON, all good')

            if not r.isReady().value:
                 for i in range(attempts):
                    print('Kicker not ready, resetting - attempt {}'.format(i))                               
                    try:                    
                        z.setDevice('N&IKPSV','RESET')
                        time.sleep(0.1)
                        r = z.readDevice('N:IKPSV.STATUS')
                        print('N:IKPSV.STATUS', r)
                        if r.isReady().value: 
                            print('Kicker successfully readied')
                            break
                        else:
                            print('FAIL - Kicker not ready')
                        if i == attempts-1:
                            print('Out of attempts!')
                            raise Exception()
                    except:
                        print('===EXTREME WARNING - KICKER RECOVERY EXCEPTION===')
            else:
                print('Kicker already READY...weird...')
                
            print('Skipping kicker setting reset!')
            time.sleep(0.1)
            z.enableSettings(False,1)    
            return status
        pass  

In [13]:
def InjectChip(z, kv):
    status = False
    fired = False
    kick_set = False
    kv = float(kv)
    if kv <= 4.0 or kv >= 4.3:
        print('Incorrect V kick magnitude - {} KV'.format(kv))
        return False
    
    try:  
        z.enableSettings(True, 5)
        # Close shutter
        if not z.readDevice('N:LGXS.STATUS').isOn().value:
            print('open shutter')
            for i in range(10):
                r = z.setDevice('N&LGXS','ON');
                print('N&LGXS', r)
                time.sleep(1) 
                r = z.readDevice('N|LGXS');
                print('N|LGXS', r)
                if r.isOn().value:
                    print('Shutter open!')
                    break
                else:
                    print('Shutter did not open - attempt #:', i)
                    if i == 10:
                        print('Aborting!')
                        raise Exception()
        else:
            print('Shutter is already open')  
        time.sleep(0.01)
        
#         if z.readDevice('N:ASTCDC.STATUS').isOn().value:
#             print('Turning on CDC')
#             for i in range(10):
#                 r = z.setDevice('N&ASTCDC','ON');
#                 print('N&ASTCDC', r)
#                 time.sleep(0.5) 
#                 r = z.readDevice('N|ASTCDC');
#                 print('N|ASTCDC', r)
#                 if r.isOn().value:
#                     print('CDC on!')
#                     break
#                 else:
#                     print('CDC did not turn on - attempt #:', i)
#                     if i == 10:
#                         print('Aborting!')
#                         raise Exception()
#         else:
#             print('CDC is already on')        
#        time.sleep(0.01)

        # Kicker 
        print('--Setting kicker')
        desired_kicker_setpt = kv
        kick_set = True;
        r = z.readDevice('N:IKPSV.STATUS')
        print('N:IKPSV.STATUS', r)
        if not r.isOn().value:
            raise Exception();
        if not r.isReady().value:
            z.setDevice('N&IKPSV','RESET')
            print('Kicker not ready - resetting!')
            time.sleep(0.5);
            r = z.readDevice('N:IKPSV.STATUS')
            print('New status:', r)
            if not r.isReady().value:
                raise Exception
        
        r = z.setDevice('N_IKPSV', desired_kicker_setpt)
        time.sleep(0.1);
        r = z.readDevice('N:IKPSV')
        print('N:IKPSV', r)
        r = r.doubleValue()
        # Experimentally determined setpoint bound
        if r < desired_kicker_setpt-0.15 or r > desired_kicker_setpt+0.03:
            raise Exception('Setpoing not within desired range')
        print('Kicker set - readback {}'.format(r))
        time.sleep(0.1);
                
        # Trigger chip thing
        print('--CHIP TRIGGER')
        r = z.setDevice('N_IBEAMR',0)
        
        print('--AWAIT A6')
        r = z.readDevice('N:LGA1DI@e,A6')
        print('N:LGA1DI@e,A6', r)    
        #time.sleep(5);
        status = True
    except Exception as e:
        print('ERROR!!!')
        print(e)
    finally:        
        # Kicker restore
        if kick_set:
            restoreKickerState(z)
                
        print('Skipping kicker setting reset!')
        z.enableSettings(False,1)    
        return status

In [14]:
# def restoreCDCON(z):
#     print('Skipping CDC restore!')
#     print('--Restoring CDC') 
#     attempts = 10
#     if not z.readDevice('N:ASTCDC.STATUS').isOn().value:
#         for i in range(attempts):
#             print('Turning on CDC - attempt {}'.format(i))                               
#             try:                    
#                 r = z.setDevice('N&ASTCDC','ON');
#                 time.sleep(0.1)
#                 r = z.readDevice('N:ASTCDC.STATUS')
#                 print('N:ASTCDC.STATUS', r)
#                 if r.isOn().value: 
#                     print('CDC on!')
#                     break
#                 else:
#                     print('FAIL - CDC still off')
#                     time.sleep(0.5)
#                 if i == attempts-1:
#                     print('Out of attempts!')
#                     raise Exception()
#             except:
#                 print('===EXTREME WARNING - CDC RECOVERY EXCEPTION===')
#     time.sleep(0.1)

In [15]:
def restoreKickerState(z):
    print('--Restoring kicker state')
    attempts = 10
    try:
        r = z.readDevice('N:IKPSV.STATUS')
        print('N:IKPSV.STATUS', r)
    except Exception as e:
        raise e

    if not r.isOn().value:
        print('===KICKER IS NOT ON - SOMETHING IS WRONG===')
        raise Exception()
    else:
        print('Kicker still ON, all good')

    if not r.isReady().value:
         for i in range(attempts):
            print('Kicker not ready, resetting - attempt {}'.format(i))                               
            try:                    
                z.setDevice('N&IKPSV','RESET')
                time.sleep(0.1)
                r = z.readDevice('N:IKPSV.STATUS')
                print('N:IKPSV.STATUS', r)
                if r.isReady().value: 
                    print('Kicker successfully readied')
                    break
                else:
                    print('FAIL - Kicker not ready')
                if i == attempts-1:
                    print('Out of attempts!')
                    raise Exception()
            except:
                print('===EXTREME WARNING - KICKER RECOVERY EXCEPTION===')
    else:
        print('Kicker already READY...weird...')

In [16]:
def restoreKickerValue(z, v):
    initial_kicker_setpt = v
    attempts = 10
    for i in range(attempts):
        try:
            z.enableSettings(True,1)
            print('Setting kicker back to {} - attempt {}'.format(initial_kicker_setpt, i))
            r = z.setDevice('N_IKPSV', initial_kicker_setpt)
            print('N_IKPSV', r)
            time.sleep(0.1)

            # Note - we are reading setpoint!
            r = z.readDevice('N_IKPSV')
            print('N_IKPSV', r)
            r2 = z.readDevice('N:IKPSV')
            print('N:IKPSV', r2)
            if np.abs(r.doubleValue()-initial_kicker_setpt) < 0.05 and np.abs(r2.doubleValue()-initial_kicker_setpt) < 0.2:
                print('Kicker reset - setpoint {}, readback {}'.format(r, r2))
                break
            else:
                print('FAIL - kicker out of tolerance - setpoint {}, readback {}'.format(r, r2))
            if i == attempts-1:
                print('Out of attempts!')
                raise Exception()
        except Exception as e:
            print(e)
            print('===EXTREME WARNING - KICKER RECOVERY EXCEPTION===')
        finally:
            z.enableSettings(False,1)

In [17]:
def VKickChip(z, kv):
    status = False
    fired = False
    kick_set = False
    kv = float(kv)
    if kv <= 0 or kv >= 1.0:
        print('Incorrect V kick magnitude - {} KV'.format(kv))
        return False
    
    try:  
        z.enableSettings(True, 5)
#         # Close shutter
#         if z.readDevice('N:ASTCDC.STATUS').isOn().value:
#             print('Turning off CDC')
#             for i in range(10):
#                 r = z.setDevice('N&ASTCDC','OFF');
#                 print('N&ASTCDC', r)
#                 time.sleep(1) 
#                 r = z.readDevice('N|ASTCDC');
#                 print('N|ASTCDC', r)
#                 if not r.isOn().value:
#                     print('CDC off!')
#                     break
#                 else:
#                     print('CDC did not turn off - attempt #:', i)
#                     if i == 10:
#                         print('Aborting!')
#                         raise Exception()
#         else:
#             print('CDC is already off')        
#         time.sleep(0.1)

        # Kicker 
        print('--Setting kicker')
        r = z.readDevice('N:IKPSV')
        print('N:IKPSV', r)
        r = r.doubleValue()
#         if r < 4.0 or r > 4.3:
#             raise Exception()
        r = z.readDevice('N_IKPSV')
        print('N_IKPSV', r)
        r = r.doubleValue()
#         if r < 4.0 or r > 4.3:
#             raise Exception()
        initial_kicker_setpt = r;
        desired_kicker_setpt = kv;
        time.sleep(0.1);

        kick_set = True;
        r = z.setDevice('N_IKPSV', desired_kicker_setpt)
        time.sleep(0.1);
        r = z.readDevice('N:IKPSV')
        print('N:IKPSV', r)
        r = r.doubleValue()
        if r < desired_kicker_setpt-0.03 or r > desired_kicker_setpt+0.03:
            raise Exception()
        print('Kicker set - readback {}'.format(r))
        time.sleep(0.1);

        r = z.readDevice('N:IKPSV.STATUS')
        print('N:IKPSV.STATUS', r)
        if not r.isOn().value:
            raise Exception();
        if not r.isReady().value:
            z.setDevice('N&IKPSV','RESET')
            print('Kicker not ready - resetting!')
            time.sleep(1);
            r = z.readDevice('N:IKPSV.STATUS').isReady()
            print('New status:', r)
            if not r.isReady().value:
                raise Exception
                
        # Trigger chip thing
        print('--CHIP TRIGGER')
        r = z.setDevice('N_IBEAMR',0)
        
        print('--AWAIT A6')
        r = z.readDevice('N:LGA1DI@e,A6')
        print('N:LGA1DI@e,A6', r)    
        #time.sleep(5);

    #     r = z.setDevice('N&IBPSTATD',"Arm Injection")
    #     print(r)
        status = True
    except Exception as e:
        print('ERROR!!!')
        print(e)

    finally:
        print('--Restoring CDC') 
        attempts = 10
        if not z.readDevice('N:ASTCDC.STATUS').isOn().value:
            for i in range(attempts):
                print('Turning on CDC - attempt {}'.format(i))                               
                try:                    
                    r = z.setDevice('N&ASTCDC','ON');
                    time.sleep(0.1)
                    r = z.readDevice('N:ASTCDC.STATUS')
                    print('N:ASTCDC.STATUS', r)
                    if r.isOn().value: 
                        print('CDC on!')
                        break
                    else:
                        print('FAIL - CDC still off')
                        time.sleep(0.5)
                    if i == attempts-1:
                        print('Out of attempts!')
                        raise Exception()
                except:
                    print('===EXTREME WARNING - CDC RECOVERY EXCEPTION===')
        time.sleep(0.1)
        
        # Kicker restore
        if kick_set:
            print('--Restoring kicker')
            attempts = 10
            dev = 0.2 #kV
            try:
                r = z.readDevice('N:IKPSV.STATUS')
                print('N:IKPSV.STATUS', r)
            except Exception as e:
                raise e

            if not r.isOn().value:
                print('===KICKER IS NOT ON - SOMETHING IS WRONG===')
                raise Exception()
            else:
                print('Kicker still ON, all good')

            if not r.isReady().value:
                 for i in range(attempts):
                    print('Kicker not ready, resetting - attempt {}'.format(i))                               
                    try:                    
                        z.setDevice('N&IKPSV','RESET')
                        time.sleep(0.1)
                        r = z.readDevice('N:IKPSV.STATUS')
                        print('N:IKPSV.STATUS', r)
                        if r.isReady().value: 
                            print('Kicker successfully readied')
                            break
                        else:
                            print('FAIL - Kicker not ready')
                        if i == attempts-1:
                            print('Out of attempts!')
                            raise Exception()
                    except:
                        print('===EXTREME WARNING - KICKER RECOVERY EXCEPTION===')
            else:
                print('Kicker already READY...weird...')
                
            print('Skipping kicker setting reset!')
#             for i in range(attempts):
#                 try:
#                     print('Setting kicker back to {} - attempt {}'.format(initial_kicker_setpt, i))
#                     r = z.setDevice('N_IKPSV', initial_kicker_setpt)
#                     print('N_IKPSV', r)
#                     time.sleep(0.1)

#                     # Note - we are reading setpoint!
#                     r = z.readDevice('N_IKPSV')
#                     print('N_IKPSV', r)
#                     r2 = z.readDevice('N:IKPSV')
#                     print('N:IKPSV', r2)
#                     if np.abs(r.doubleValue()-initial_kicker_setpt) < 0.01 and np.abs(r2.doubleValue()-initial_kicker_setpt) < dev:
#                         print('Kicker reset - setpoint {}, readback {}'.format(r, r2))
#                         break
#                     else:
#                         print('FAIL - kicker out of tolerance - setpoint {}, readback {}'.format(r, r2))
#                     if i == attempts-1:
#                         print('Out of attempts!')
#                         raise Exception()
#                 except: 
#                     print('===EXTREME WARNING - KICKER RECOVERY EXCEPTION===')
            time.sleep(0.1)
            z.enableSettings(False,1)    
            return status
        pass  